### `qm-data` Data Viewer
#### Rev. 8/23/2022.

In [ ]:
import au
import h5py
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

# data directory
run_path = '/media/ayounis/DATA1/qray/Untitled-paper/run5'
# spatial/temporal stride
sr = 10; st = 1;

# read input deck parameters
fin = open(run_path+'/input.deck','r')
if ('R0' in globals()): del R0
if ('R0s' in globals()): del R0s
for ln in fin:
    if (ln[0:4] == 'mode'): sln = ln.split(); mode = str(sln[1][0:4]); # simulation mode
    if (ln[0:2] == 'dm'): sln = ln.split(); dm = float(sln[1]); # masking distance
    if (ln[0:2] == 'ar'): sln = ln.split(); ar = float(sln[1]); # absorbing radius (legacy)
    if (ln[0:4] == 'omg0'): sln = ln.split(); omg0 = float(sln[1]); # frequency
    if (ln[0:4] == 't_on'): sln = ln.split(); t_on = float(sln[1]); # pulse start time
    if (ln[0:2] == 'R0' and 'R0' not in globals()): sln = ln.split(); R0 = float(sln[1]); # ring radius
    if (ln[0:2] == 'R0' and 'R0' in globals()): sln = ln.split(); R0s = float(sln[1]); # t-SURFF integration radius
fin.close()
T0 = 2*np.pi/omg0 # optical period

### Load data

In [ ]:
f = h5py.File(run_path+'/out/qm_data.h5','r')
#print(list(f.keys()))

'''load spatio-temporal mesh'''
t = np.array(f['t'][:,0]); nt = len(t); dt = t[1]-t[0];

tf = np.array(f['tf'][::st,0]); ntf = len(tf); dtf = tf[1]-tf[0];
tp = np.array(f['tp'][:,0]); ntp = len(tp); dtp = tp[1]-tp[0];
x = np.array(f['x'][::sr,0]); nx = len(x); dx = x[1]-x[0];
y = np.array(f['y'][::sr,0]); ny = len(y); dy = y[1]-y[0];
xx,yy = np.meshgrid(x,y)

'''load electric field'''
Et = np.array(f['Ert']).transpose()
if (mode == '1d2e'): Et = Et[0,:];

'''load expectation value quantities'''
norm = np.array(f['norm(psi)'][:,0])
enrg = np.array(f['enrg(psi)'][:,0])

'''load wavefunction (volumetric)'''
psi = np.array(f['Re(psi)'][::st,::sr,::sr]) + 1j*np.array(f['Im(psi)'][::st,::sr,::sr])
psi = psi.transpose(1,2,0)
mem_psi = psi.size*psi.itemsize # memory (bytes)

'''load wavefunction (projection)'''
psi_p = np.array(f['Re(p-psi)']) + 1j*np.array(f['Im(p-psi)'])
psi_p = psi_p.transpose(2,1,0)
mem_psi_p = psi_p.size*psi_p.itemsize # memory (bytes)

print('dim(psi) (ny/'+str(sr)+', nx/'+str(sr)+', nt/'+str(st)+') = ' + str(psi.shape) + ', size: ' + str(np.round(mem_psi/(1024**3),3)) + ' GB')
print('dim(p-psi) (nt, nx, cm) = ' + str(psi_p.shape) + ', size: ' + str(np.round(mem_psi_p/(1024**3),3)) + ' GB')

'''calculate log10 psi2'''
log10_psi2 = np.ma.log10(abs(psi)**2).filled(-20)
log10_psip = np.ma.log10(abs(psi_p)**2).filled(-20)

In [ ]:
'''load t-SURFF data'''
kx = np.array(f['kx'][:,0])
ky = np.array(f['ky'][:,0])
p_dist = np.array(f['Re(tSURFF)']) + 1j*np.array(f['Im(tSURFF)'])

### Plotter settings

In [ ]:
font = {'weight':'normal','size':26}

matplotlib.rc('font',**font)
dpi = 75 # 300

matplotlib.rcParams['font.family'] = 'sans-serif'
#matplotlib.rcParams['font.family'] = 'serif'
#matplotlib.rcParams['font.sans-serif'] = 'Arial'

extent_x = [x[0],x[-1],y[0],y[-1]]

# toggle PNG/EPS export
save_PNG = False
save_EPS = False

### Plot $\Psi(\vec{x},t)$

In [ ]:
'''PARAMETERS'''
# viewer settings
view_type = 'log'
view_mode = 'contourf'
view_cmap = 'turbo'
# use TeX for labels
usetex = False
# time-step index
it = 0
# x-y plot window
lims = [x[0]-2, x[-1]+2, y[0]-2, y[-1]+2]
# color-axis limits, levels, cbar shrink, pad, bins
# clim[1] *cannot* eq. 0 if view_mode is 'contourf'
if view_type == 'log': clim = [-6,0]
if view_type == 'lin': clim = [0,2e-3]
nzs = 100; shrink = 0.75; pad = 0.02; nbins = 10;
'''axes overlay'''
axoo = True # toggle
axow = 1.0 # width
axoc = 'k' # color
'''e-force vector overlay'''
efvo = False # toggle
efvs = 10.0 # length
efvw = 0.5 # width
efvc = 'w' # color
'''VD/t-SURFF/absorption region overlays'''
vdao = True # toggle
vdaw = 1.5 # width
vdls = ['-','-','-'] # line styles
vdac = ['k','k','r'] # colors

matplotlib.rc('text',**{'usetex':usetex})
plt.figure(figsize=(11,9), dpi=dpi, facecolor='w')
extent = extent_x

if (view_mode == 'contourf'):
    if (view_type == 'log'):
        plt.contourf(xx, yy, log10_psi2[:,:,it], levels=np.linspace(clim[0],clim[1],nzs), cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad, ticks=np.flip(np.arange(clim[0],clim[1]))) # note: clim[i]>0 or else error
    elif (view_type == 'lin'):
        plt.contourf(xx, yy, abs(psi[:,:,it])**2, levels=np.linspace(clim[0],clim[1],nzs), cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad)

elif (view_mode == 'imshow'):
    if (view_type == 'log'):
        plt.imshow(log10_psi2[:,:,it], extent=extent, origin='lower', cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad, ticks=np.flip(np.arange(clim[0],clim[1])))
    elif (view_type == 'lin'):
        plt.imshow(abs(psi[:,:,it])**2, extent=extent, origin='lower', cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad)
    plt.clim([clim[0],clim[1]])

cbar.ax.locator_params(nbins=nbins)

if (view_type == 'log'):
    cbar.set_label('$\\log|\\Psi(\\vec{x},t)|^2$', labelpad=10, fontsize=font['size']/1.25, usetex=usetex)
elif (view_type == 'lin'):
    cbar.set_label('$|\\Psi(\\vec{x},t)|^2$', labelpad=10, fontsize=font['size']/1.25, usetex=usetex)
cbar.ax.tick_params(labelsize=font['size']/1.25)

if (mode == '2d1e'):
    plt.xlabel('$x$ $(a_0)$', usetex=usetex)
    plt.ylabel('$y$ $(a_0)$', labelpad=-10, usetex=usetex)
elif (mode == '1d2e'):
    plt.xlabel('$x_1$ $(a_0)$', usetex=usetex)
    plt.ylabel('$x_2$ $(a_0)$', labelpad=-10, usetex=usetex)

# overlay axes
if (axoo):
    plt.plot([lims[0],lims[-1]], [0,0], c=axoc, lw=axow)
    plt.plot([0,0], [lims[0],lims[-1]], c=axoc, lw=axow)

# overlay instantaneous electric force vector
if (efvo and mode == '2d1e'):
    plt.arrow(0, 0,
              -efvs*Et[0,int(tf[it]/dt)]/np.max(Et),
              -efvs*Et[1,int(tf[it]/dt)]/np.max(Et),
              width=efvw, color=efvc)
elif (efvo and mode == '1d2e'):
    plt.arrow(0, 0,
              -efvs*Et[int(tf[it]/dt)]/np.max(Et),
              -efvs*Et[int(tf[it]/dt)]/np.max(Et),
              width=efvw, color=efvc)

# overlay VD/t-SURFF/absorption region markers
if (vdao):
    ph = np.linspace(0, 2*np.pi, 10**3)
    plt.plot(R0*np.cos(ph), R0*np.sin(ph), ls=vdls[0], c=vdac[0], lw=vdaw)
    plt.plot(R0s*np.cos(ph), R0s*np.sin(ph), ls=vdls[1], c=vdac[1], lw=vdaw)
    if 'dm' in globals():
        plt.plot([x[0]+dm,x[0]+dm],[y[0]+dm,y[-1]-dm], ls=vdls[2], c=vdac[2], lw=vdaw)
        plt.plot([x[0]+dm,x[-1]-dm],[y[-1]-dm,y[-1]-dm], ls=vdls[2], c=vdac[2], lw=vdaw)
        plt.plot([x[-1]-dm,x[-1]-dm],[y[-1]-dm,y[0]+dm], ls=vdls[2], c=vdac[2], lw=vdaw)
        plt.plot([x[0]+dm,x[-1]-dm],[y[0]+dm,y[0]+dm], ls=vdls[2], c=vdac[2], lw=vdaw)
    if 'ar' in globals():
        plt.plot(ar*np.cos(ph), ar*np.sin(ph), ls=vdls[2], c=vdac[2], lw=vdaw)

# stamp time
plt.text(lims[0], lims[1], 't = ' + str(np.round(tf[it]*au.t*1e+15,2)) + ' fs ('
         + str(np.round((tf[it]-t_on)/T0,2)) + ' cyc.)',
         fontsize=font['size']/1.25, c='k', ha='left', va='bottom', usetex=usetex);

plt.xlim([lims[0],lims[1]])
plt.ylim([lims[2],lims[3]])

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
ax.tick_params(which='major', direction='in', size=8)
ax.tick_params(which='minor', direction='in', size=4)
ax.tick_params(axis='x', which='both', bottom=True, top=True)
ax.tick_params(axis='y', which='both', left=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(50))
ax.xaxis.set_minor_locator(MultipleLocator(25))
ax.yaxis.set_major_locator(MultipleLocator(50))
ax.yaxis.set_minor_locator(MultipleLocator(25))

plt.tight_layout()
if (save_PNG): plt.savefig(view_type+'-psi2_it'+str(it)+'.png', dpi=dpi)
if (save_EPS): plt.savefig(view_type+'-psi2_it'+str(it)+'.eps', format='eps')
plt.show()

### Plot t-SURFF momentum probability distribution

In [ ]:
plt.figure(figsize=(7,7), dpi=dpi)
plt.imshow(np.log10(abs(p_dist)**2), extent=[kx[0],kx[-1],ky[0],ky[-1]], aspect=1., origin='lower', cmap='hot')
#plt.xlim([-1,1])
#plt.ylim([-1,1])
#plt.imshow(abs(p_dist), extent=[kx[0],kx[-1],ky[0],ky[-1]], aspect=1., origin='lower', cmap='turbo')

'''
ph = np.linspace(0, 2*np.pi, 10**3)
for k in range(10):
    plt.plot(0.3797*k*np.sin(ph), 0.3797*k*np.cos(ph), '--', c='w')
'''

plt.colorbar(shrink=0.5)
plt.clim([-6,0])
#plt.clim([0,1e-20])
plt.tight_layout()
plt.show()

### Calculate $\Phi(\vec{p},t)$

In [ ]:
# time-step index
it = -1
print('t = ' + str(np.round(f['tf'][it][0],3)) + ' au (' + str(np.round(f['tf'][it][0]/T0,3)) + ' cyc)')

psi_it = np.array(f['Re(psi)'][it,:,:]) + 1j*np.array(f['Im(psi)'][it,:,:])
x1 = np.array(f['x'][:,0])
y1 = np.array(f['y'][:,0])

'''Make & visualize bound-state wavefunction filter'''
# size parameters
Rc = 500; Delta = 8;
sig_x = 200; sig_y = 200;

xx1,yy1 = np.meshgrid(x1,y1)
Ms = (1-np.exp(-(xx1/sig_x)**2))*(1-np.exp(-(yy1/sig_y)**2))
#Ms = 1/(1 + np.exp(-np.sqrt(xx1**2 + yy1**2)+Rc/Delta))

plt.figure(figsize=(3,3))
plt.imshow(Ms, extent=extent_x, origin='lower', aspect=1, cmap='gray', vmin=0, vmax=1)
#plt.imshow(np.ma.log10(abs(Ms*psi_it)**2), extent=extent_x, origin='lower', aspect=1, cmap='turbo', vmin=-9, vmax=0)
plt.colorbar()
plt.show()

px = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(len(x1),x1[1]-x1[0])); dpx = px[1]-px[0];
py = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(len(y1),y1[1]-y1[0])); dpy = py[1]-py[0];
ppxx,ppyy = np.meshgrid(px,py)
extent_p = [px[0],px[-1],py[0],py[-1]]

phi = np.fft.fftshift(np.fft.fft2(Ms*psi_it))

'''normalize
for it in range(ntf):
    phi /= np.sqrt(dpx*dpy*np.trapz(np.trapz(abs(phi)**2)))
'''

# calculate log10 phi2
log10_phi2 = np.ma.log10(abs(phi)**2).filled(-20)

### Plot $\Phi(\vec{p},t)$

In [ ]:
'''PARAMETERS'''
# viewer settings
view_type = 'log'
view_mode = 'imshow'
view_cmap = 'turbo'
# use TeX for labels
usetex = False
# px-py plot window
lims = [px[0]-1/4, px[-1]+1/4]
lims = [-2.5, 2.5]
# color-axis limits, levels, cbar shrink, pad, bins
if view_type == 'log': clim = [-4,4]
if view_type == 'lin': clim = [0,5]
nzs = 50; shrink = 0.9; pad = 0.02; nbins = 6;
'''axes overlay'''
axoo = False # toggle
axow = 1.0 # width
axoc = 'w' # color

matplotlib.rc('text',**{'usetex':usetex})
plt.figure(figsize=(11,9), dpi=dpi, facecolor='w')
extent = extent_p

if (view_mode == 'contourf'):
    if (view_type == 'log'):
        plt.contourf(ppxx, ppyy, log10_phi2, levels=np.linspace(clim[0],clim[1],nzs), cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad, ticks=np.flip(np.arange(clim[0],clim[1])));
    elif (view_type == 'lin'):
        plt.contourf(ppxx, ppyy, abs(phi)**2, levels=np.linspace(clim[0],clim[1],nzs), cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad)

elif (view_mode == 'imshow'):
    if (view_type == 'log'):
        plt.imshow(log10_phi2, extent=extent, origin='lower', cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad, ticks=np.flip(np.arange(clim[0],clim[1])));
    elif (view_type == 'lin'):
        plt.imshow(abs(phi)**2, extent=extent, origin='lower', cmap=view_cmap)
        cbar = plt.colorbar(shrink=shrink, pad=pad)
    plt.clim([clim[0],clim[1]])

cbar.ax.locator_params(nbins=nbins)

if (view_type == 'log'):
    cbar.set_label('$\\log_{10}|\\Phi(\\vec{p},t)|^2$', labelpad=10, fontsize=font['size'], usetex=usetex)
elif (view_type == 'lin'):
    cbar.set_label('$|\\Phi(\\vec{p},t)|^2$', labelpad=10, fontsize=font['size'], usetex=usetex)
cbar.ax.tick_params(labelsize=font['size'])

'''
for n in range(6):
    plt.plot(n*omg0*np.cos(ph)+0.1, n*omg0*np.sin(ph)+0.1, '--', lw=1, c='w')
'''

if (mode == '2d1e'):
    plt.xlabel('$p_x$ (a.u.)', usetex=usetex)
    plt.ylabel('$p_y$ (a.u.)', labelpad=0, usetex=usetex)
elif (mode == '1d2e'):
    plt.xlabel('$p_1$ (a.u.)', usetex=usetex)
    plt.ylabel('$p_2$ (a.u.)', labelpad=0, usetex=usetex)

# overlay axes
if (axoo):
    plt.plot([lims[0],lims[-1]], [0,0], c=axoc, lw=axow)
    plt.plot([0,0], [lims[0],lims[-1]], c=axoc, lw=axow)

# stamp time
'''
plt.text(0, lims[1], 't = ' + str(np.round(tf[it]*au.t*1e+15,2)) + ' fs ('
         + str(np.round((tf[it]-t_on)/T0,2)) + ' c.)',
         fontsize=font['size'], c='k', ha='center', va='bottom', usetex=usetex);
'''

plt.xlim([lims[0],lims[1]])
plt.ylim([lims[0],lims[1]])

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
ax.tick_params(which='major', direction='in', size=12, width=1.5, color='w')
ax.tick_params(which='minor', direction='in', size=6, width=1.5, color='w')
ax.tick_params(axis='x', which='both', bottom=True, top=True)
ax.tick_params(axis='y', which='both', left=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.tight_layout()
if (save_PNG): plt.savefig(view_type+'-phi2_it'+str(it)+'.png', dpi=dpi)
if (save_EPS): plt.savefig(view_type+'-phi2_it'+str(it)+'.eps', format='eps')
plt.show()

### Plot $\Psi(\vec{x},t)$ projection

In [ ]:
'''PARAMETERS'''
# viewer settings
view_cmap = 'turbo'
# use TeX for labels
usetex = False
# color-axis limits
clim = [-3,0]

matplotlib.rc('text',**{'usetex':usetex})
fig, axs = plt.subplots(1, 2, figsize=(11,9), dpi=dpi)
extent = [x[0],-x[0],tp[0]/T0,tp[-2]/T0]; aspect = 0.8*(x[-1]-x[0])/(tp[-2]-tp[0])*T0;

axs[0].imshow(log10_psip[:,:,0], extent=extent, aspect=aspect, origin='lower', cmap=view_cmap, clim=clim)
axs[1].imshow(log10_psip[:,:,1], extent=extent, aspect=aspect, origin='lower', cmap=view_cmap, clim=clim)

axs[0].set_xlabel('$x_1$ $(a_0)$', usetex=usetex), axs[0].set_ylabel('Time ($T_0$)', usetex=usetex, labelpad=5)
axs[1].set_xlabel('$x_2$ $(a_0)$', usetex=usetex)

axs[0].set_xlim([-20,200])

for i in range(2):
    axs[i].xaxis.set_major_locator(MultipleLocator(100))
    axs[i].xaxis.set_minor_locator(MultipleLocator(100/4))
    axs[i].yaxis.set_major_locator(MultipleLocator(2))
    axs[i].yaxis.set_minor_locator(MultipleLocator(1))

plt.tight_layout()
plt.show()

### Plot norm / energy

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,7), dpi=dpi, facecolor='w')

# shift/normalize time
tn = (t-t_on)/T0

ln1 = ax1.plot(tn, enrg, 'mediumblue', lw=2.0);
ax1.spines['left'].set_color('mediumblue');
ax1.tick_params(axis='y', colors='mediumblue');
plt.ylabel('$\\langle H(\\vec{x},\\vec{p})\\rangle$ (a.u.)', c='mediumblue', labelpad=10);

ax2 = ax1.twinx();
ln2 = ax2.plot(tn, norm, 'k', lw=2.0);
plt.ylabel('$\int|\\Psi(\\vec{x},t)|^2\\,d^2x$', labelpad=15);

ax1.set_xlabel('Time ($T_0$)', labelpad=10);
ax1.set_xlim([tn[0],tn[-1]]);

plt.tight_layout()
#plt.savefig('expvals.png', dpi=dpi)
#plt.savefig('expvals.eps', format='eps')
plt.show()

In [ ]:
plt.plot(t, np.array(f['Ert'])[:,0])
plt.plot(t, np.array(f['Art'])[:,0])
plt.plot(t, np.array(f['Crt'])[:,0])